In [1]:
# !pip install -q copyleaks

In [11]:


import base64
import random
from copyleaks.copyleaks import Copyleaks
from copyleaks.exceptions.command_error import CommandError
from copyleaks.models.submit.ai_detection_document import NaturalLanguageDocument, SourceCodeDocument
from copyleaks.models.submit.document import FileDocument, UrlDocument, OcrFileDocument
from copyleaks.models.submit.properties.scan_properties import ScanProperties
from copyleaks.models.export import *
from copyleaks.models.submit.score_weights import ScoreWeights
from copyleaks.models.submit.writing_assistant_document import WritingAssistantDocument
# Register on https://api.copyleaks.com and grab your secret key (from the dashboard page).


In [12]:

EMAIL_ADDRESS = 'liviaellen@msn.com'
KEY = '68f00127-8e08-48ef-9e41-7f55668a99d3'


In [13]:

try:
    auth_token = Copyleaks.login(EMAIL_ADDRESS, KEY)
except CommandError as ce:
    response = ce.get_response()
    print(f"An error occurred (HTTP status code {response.status_code}):")
    print(response.content)
    exit(1)

print("Logged successfully!\nToken:")
print(auth_token)


Logged successfully!
Token:
{'access_token': '98F183EA3E227BD640159EC9222021C7BCC20849A617B54E790B0CC70017AE9F', '.issued': '2024-09-22T00:06:48.2735295Z', '.expires': '2024-09-24T00:06:48.2735296Z'}


In [5]:

# This example is going to scan a FILE for plagiarism.
# Alternatively, you can scan a URL using the class `UrlDocument`.

print("Submitting a new file...")
BASE64_FILE_CONTENT = base64.b64encode(b'Hello world').decode('utf8')  # or read your file and convert it into BASE64 presentation.
FILENAME = "hello.txt"
scan_id = random.randint(100, 100000)  # generate a random scan id
file_submission = FileDocument(BASE64_FILE_CONTENT, FILENAME)
# Once the scan completed on Copyleaks servers, we will trigger a webhook that notify you.
# Write your public endpoint server address. If you testing it locally, make sure that this endpoint
# is publicly available.
scan_properties = ScanProperties('https://your.server/webhook?event={{STATUS}}')
scan_properties.set_sandbox(True)  # Turn on sandbox mode. Turn off on production.
file_submission.set_properties(scan_properties)
Copyleaks.submit_file(auth_token, scan_id, file_submission)  # sending the submission to scanning
print("Send to scanning")
print("You will notify, using your webhook, once the scan was completed.")


Submitting a new file...
Send to scanning
You will notify, using your webhook, once the scan was completed.


In [6]:

# This example is going to use the AI detector client to detect ai in text
sample_text = "Lions are social animals, living in groups called prides, typically consisting of several females, their offspring, and a few males. Female lions are the primary hunters, working together to catch prey. Lions are known for their strength, teamwork, and complex social structures."
natural_language_submission = NaturalLanguageDocument(sample_text)
natural_language_submission.set_sandbox(True)
response = Copyleaks.AiDetectionClient.submit_natural_language(auth_token, scan_id, natural_language_submission)
print(response)


{'scanType': 'sandbox', 'modelVersion': 'v4', 'results': [{'classification': 2, 'probability': 0.9714061, 'matches': [{'text': {'chars': {'starts': [0], 'lengths': [1557]}, 'words': {'starts': [0], 'lengths': [271]}}}]}, {'classification': 1, 'probability': 0.006316364, 'matches': [{'text': {'chars': {'starts': [1557], 'lengths': [135]}, 'words': {'starts': [271], 'lengths': [27]}}}]}, {'classification': 2, 'probability': 0.32940686, 'matches': [{'text': {'chars': {'starts': [1692], 'lengths': [87]}, 'words': {'starts': [298], 'lengths': [17]}}}]}, {'classification': 1, 'probability': 0.9098962, 'matches': [{'text': {'chars': {'starts': [1779], 'lengths': [532]}, 'words': {'starts': [315], 'lengths': [85]}}}]}], 'summary': {'human': 0.28, 'ai': 0.72}, 'scannedDocument': {'scanId': 'example-id', 'totalWords': 400, 'totalExcluded': 0, 'actualCredits': 0, 'expectedCredits': 1, 'creationTime': '2023-03-08T11:40:03.0232297Z'}}


In [7]:


# This example is going to use the AI detector client to detect ai in source code
sample_code = (
    "def add(a, b):\n"
    "    return a + b\n"
    "\n"
    "def multiply(a, b):\n"
    "    return a * b\n"
    "\n"
    "def main():\n"
    "    x = 5\n"
    "    y = 10\n"
    "    sum_result = add(x, y)\n"
    "    product_result = multiply(x, y)\n"
    "    print(f'Sum: {sum_result}')\n"
    "    print(f'Product: {product_result}')\n"
    "\n"
    "if __name__ == '__main__':\n"
    "    main()"
)
source_code_submission = SourceCodeDocument(sample_code, "example.py")
source_code_submission.set_sandbox(True)
response = Copyleaks.AiDetectionClient.submit_natural_language(auth_token, scan_id, source_code_submission)
print(response)


{'scanType': 'sandbox', 'modelVersion': 'v4', 'results': [{'classification': 2, 'probability': 0.9714061, 'matches': [{'text': {'chars': {'starts': [0], 'lengths': [1557]}, 'words': {'starts': [0], 'lengths': [271]}}}]}, {'classification': 1, 'probability': 0.006316364, 'matches': [{'text': {'chars': {'starts': [1557], 'lengths': [135]}, 'words': {'starts': [271], 'lengths': [27]}}}]}, {'classification': 2, 'probability': 0.32940686, 'matches': [{'text': {'chars': {'starts': [1692], 'lengths': [87]}, 'words': {'starts': [298], 'lengths': [17]}}}]}, {'classification': 1, 'probability': 0.9098962, 'matches': [{'text': {'chars': {'starts': [1779], 'lengths': [532]}, 'words': {'starts': [315], 'lengths': [85]}}}]}], 'summary': {'human': 0.28, 'ai': 0.72}, 'scannedDocument': {'scanId': 'example-id', 'totalWords': 400, 'totalExcluded': 0, 'actualCredits': 0, 'expectedCredits': 1, 'creationTime': '2023-03-08T11:40:03.0232297Z'}}


In [10]:

# This example is going to use the WritingAssistant client to get feedback on text
score_weight = ScoreWeights()
score_weight.set_grammar_score_weight(0.2)
score_weight.set_mechanics_score_weight(0.3)
score_weight.set_sentence_structure_score_weight(0.5)
score_weight.set_word_choice_score_weight(0.4)
submission = WritingAssistantDocument(sample_text)
submission.set_score(score_weight)
submission.set_sandbox(True)
print(auth_token, scan_id, submission)
try:
    response = Copyleaks.WritingAssistantClient.submit_text(auth_token, scan_id, submission)
    print(response)
except Exception as E:
    print('error', E)
    print("skipping writing Assistant")


{'access_token': '4A5F9C6863EF87AEEB52FCA35F5830CB74B8FC840B980E3D5B97B57A6C09623D', '.issued': '2024-09-21T23:44:43.7886617Z', '.expires': '2024-09-23T23:44:43.7886617Z'} 46389 <copyleaks.models.submit.writing_assistant_document.WritingAssistantDocument object at 0x110766ab0>
error <Response [400]>
skipping writing Assistant


In [9]:

# # Wait for completion webhook arrival...
# # Read more: https://api.copyleaks.com/documentation/v3/webhooks
# # Uncomment the following code to create an export task:
# # # Once the webhooks arrived and the scan was completed successfully (see the `status` flag) you can
# # # proceed to exporting all the artifacts related to your scan.
# export = Export()
# export.set_completion_webhook('https://your.server/webhook/export/completion')
# crawled = ExportCrawledVersion()  # Uncomment if you want to download the crawled version of your submitted document.
# crawled.set_endpoint('https://your.server/webhook/export/crawled')
# crawled.set_verb('POST')
# crawled.set_headers([['key', 'value'], ['key2', 'value2']])  # optional
# export.set_crawled_version(crawled)

# # # For each of the results in the Completed Webhook, you will get a unique `id`.
# # # In the following example we will export 2 results from Copyleaks's servers:
# results1 = ExportResult()
# results1.set_id('2b42c39fba')  # change with your result id
# results1.set_endpoint('https://your.server/webhook/export/result/2b42c39fba')
# results1.set_verb('POST')
# results1.set_headers([['key', 'value'], ['key2', 'value2']])

# results2 = ExportResult()
# results2.set_id('08338e505d')  # change with your result id
# results2.set_endpoint('https://your.server/webhook/export/result/08338e505d')
# results2.set_verb('POST')
# results2.set_headers([['key', 'value'], ['key2', 'value2']])

# export.set_results([results1, results2])

# Copyleaks.export(auth_token, scan_id, 'export-id', export)  # 'export-id' value determind by you.

# # Wait while Copyleaks servers exporting artifacts...
# # Once process completed, you will get the "Export Completed" webhook.
# # Read more: https://api.copyleaks.com/documentation/v3/webhooks/export-completed

# # # For Repositories:
# repo = SearchRepository()
# repo.set_include_my_submissions(True)
# repo.set_include_others_submissions(True)
# repo.set_id("ID_FETCHED_DASHBOARD")
# scan_properties.set_scanning(Scanning().set_repositories(repo))

# # # generate a pdf report:
# pdf = Pdf() # Creating instance of Pdf.
# pdf.set_create(True) # Setting the create pdf to True to generate PDF report.
# scan_properties.set_pdf(pdf) # Will generate PDF report.
